# Generating Results for Project 2

This notebook includes all the code needed to run the calculations and simulations that I used in my second design project. This notebook also has the capability of simulating 2 two robots at the same time.
The notebook will first go through the calculations for LQR and then go through the simulation and results

First, we import all the required libraries

In [1]:
# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import linalg

# Given pybullet script (it is an interface to the pybullet simulator)
import ae353_segbot

import importlib
importlib.reload(ae353_segbot)

<module 'ae353_segbot' from 'c:\\Users\\anshu\\Desktop\\Class Materials\\AE 353 Git\\ae353-sp21\\projects\\02_segbot\\ae353_segbot.py'>

## Linearizing the System

First we need to setup the symbolic equations and linearize the system about an equilibrium point.

In [2]:
(e_l,
 e_h,
 v,
 w,
 theta,
 thetadot,
 tau_L,
 tau_R) = sym.symbols('e_l, e_h, v, w, theta, thetadot, tau_L, tau_R', real=True)

This large block bellow is taken mostly from the DeriveEOM notebook that was already provided. However, there were only 5 equation (theta wasn't part of the state) so I added that.

In [3]:
# Dimensions of chassis
dx = 0.4
dy = 0.6
dz = 0.8

# Distance between axle and COM of chassis
h = 0.3

# Half-distance between wheels
a = 0.7 / 2

# Mass of chassis
mb = 12.

# MOI of chassis
Jbx = (mb / 12) * (dy**2 + dz**2)
Jby = (mb / 12) * (dx**2 + dz**2)
Jbz = (mb / 12) * (dx**2 + dy**2)

# Radius of each wheel
r = 0.325

# Width of each wheel
hw = 0.075

# Mass of each wheel
mw = 1.2

# MOI of each wheel
Jw = (mw / 2) * r**2
Jwt = (mw / 12) * (3 * r**2 + hw**2)

# Total mass
m = mb + 2 * mw

# Total MOI
Jx = Jbx + 2 * Jwt
Jy = Jby
Jz = Jbz + 2 * Jwt

# Acceleration of gravity
g = 9.81

# Dimensions
h = sym.nsimplify(h)
a = sym.nsimplify(a)
r = sym.nsimplify(r)

# Masses
mb = sym.nsimplify(mb)
mw = sym.nsimplify(mw)
m = sym.nsimplify(m)

# MOIs
Jx = sym.nsimplify(Jx)
Jy = sym.nsimplify(Jy)
Jz = sym.nsimplify(Jz)
Jw = sym.nsimplify(Jw)
Jwt = sym.nsimplify(Jwt)

# Gravity
g = sym.nsimplify(g)

M = sym.Matrix([[m + 2 * Jw / r**2, 0, mb * h * sym.cos(theta)],
                [0, (Jx + mb * h**2) * sym.sin(theta)**2 + Jz * sym.cos(theta)**2 + (2 * Jw * a**2 / r**2) + 2 * mw * a**2, 0],
                [mb * h * sym.cos(theta), 0, Jy * mb * h**2]])
N = sym.Matrix([[mb * h * (w**2 + thetadot**2) * sym.sin(theta)],
                [-2 * (Jx - Jz + m * h**2) * sym.cos(theta) * sym.sin(theta) * w * thetadot - mb * h * sym.sin(theta) * v * w],
                [(Jx - Jz + mb * h**2) * sym.cos(theta) * sym.sin(theta) * w**2 + mb * g * h * sym.sin(theta)]])
R = sym.Matrix([[1 / r, 1 / r],
                [-a / r, a / r],
                [-1, -1]])
f = sym.simplify(M.inv() * (N + R * sym.Matrix([[tau_L], [tau_R]])))
# In the definition below for f, I added the 5th equation "thetadot" which is the derivative of theta, to complete my set of the equations of motion
f = sym.Matrix([[v * sym.sin(e_h)],
                [w],
                [f[0]],
                [f[1]],
                [thetadot],
                [f[2]]])

In [20]:
# See f
f

Matrix([
[                                                                                                                                                                              v*sin(e_h)],
[                                                                                                                                                                                       w],
[       -(2400*tau_L + 2400*tau_R + 2808*(thetadot**2 + w**2)*sin(theta) + 13*(250*tau_L + 250*tau_R - 195*w**2*sin(2*theta) - 8829*sin(theta))*cos(theta))/(11700*cos(theta)**2 - 12168)],
[                                                                       32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
[                                                                                                                                                                                thetadot],
[(42250*tau_L + 42250*tau_R - 32955*w**2*sin(2*thet

In [5]:
# Make f an executable function
f_num = sym.lambdify((e_l, e_h, v, w, theta, thetadot, tau_L, tau_R), f)

In [31]:
# Equilibrium values around which the system is linearized

e_l_e = 0
e_h_e = 0
v_e = 4 # Around the velocity that we want, can change this if needed
w_e = 0
theta_e = 0
thetadot_e = 0
tau_L_e = 0
tau_R_e = 0

In [32]:
# Checking if the equilibrium values work
f_eq = f_num(e_l_e, e_h_e, v_e, w_e, theta_e, thetadot_e, tau_L_e, tau_R_e)
f_eq #evaluates to 0! Nice!

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-0.]])

Now that we have the governing equations of motion and an equilibrium position, we can linearize the system about that equilibrium point.

In [33]:
#Now we find the Jacobians
f_jacob_x = f.jacobian([e_l, e_h, v, w, theta, thetadot])
f_jacob_u = f.jacobian([tau_L,tau_R])

#And then we find functions for A and B using this jacobian
A_num = sym.lambdify((e_l, e_h, v, w, theta, thetadot, tau_L, tau_R),f_jacob_x)
B_num = sym.lambdify((e_l, e_h, v, w, theta, thetadot, tau_L, tau_R),f_jacob_u)

#Finally, we find the linearized state space model by evaluating A_num and B_num at the equilibrium points
A = A_num(e_l_e, e_h_e, v_e, w_e, theta_e, thetadot_e, tau_L_e, tau_R_e).astype(float)
B = B_num(e_l_e, e_h_e, v_e, w_e, theta_e, thetadot_e, tau_L_e, tau_R_e).astype(float)

In [34]:
# See the linear system
A

array([[ 0.00000e+00,  4.00000e+00,  0.00000e+00,  0.00000e+00,
         0.00000e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00,
         0.00000e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        -2.45250e+02, -0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
         0.00000e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
         0.00000e+00,  1.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -0.00000e+00,
         1.06275e+03, -0.00000e+00]])

In [35]:
B

array([[  0.        ,   0.        ],
       [  0.        ,   0.        ],
       [ 12.07264957,  12.07264957],
       [ -1.05014439,   1.05014439],
       [  0.        ,   0.        ],
       [-51.46011396, -51.46011396]])

## Checking Controllability

We need to check if our system is controllable or not before we start designing a controller. For this, we calculate the $$W$$ matrix and check its rank.

In [36]:
# Find the number of states
n = A.shape[0]

# Initialize W with its first column
W = B

# Create W one column at a time by iterating over i from 1 to n-1
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    W = np.block([W, col])
    

In [37]:
W

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -4.20057758e+00,  4.20057758e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.05014439e+00,
         1.05014439e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.20726496e+01,  1.20726496e+01,  0.00000000e+00,
         0.00000000e+00,  1.26205929e+04,  1.26205929e+04,
         0.00000000e+00,  0.00000000e+00,  1.34125352e+07,
         1.34125352e+07,  0.00000000e+00,  0.00000000e+00],
       [-1.05014439e+00,  1.05014439e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -5.1460114

In [38]:
# What about the rank?
print(np.linalg.matrix_rank(A))
print(np.linalg.matrix_rank(W))
# This is weird

4
6


## Finding controller gains using LQR

In [57]:
Q = np.diag([50,17.5,3,15,60,30]) # Lateral error, heading error, forward speed, turning rate, pitch, pitch rate
R = np.diag([1,1])

P = linalg.solve_continuous_are(A, B, Q, R)
K = linalg.inv(R) @  B.T @ P


In [58]:
K.tolist()

[[-5.000000000000083,
  -13.748174838241393,
  -1.0000000000000284,
  -4.5378077845712355,
  -23.195157084954516,
  -4.172339830994215],
 [4.99999999999994,
  13.748174838241162,
  -1.0000000000000284,
  4.537807784571223,
  -23.19515708495466,
  -4.172339830994215]]

In [41]:
# Check stability of the K calculated above
F  = A - B@K
s = linalg.eigvals(F)
print(s.real)

[-4.01980931e+02 -5.23323757e+00 -1.55568784e+00 -1.55568784e+00
 -2.88006796e+00 -3.62113705e-01]


In [42]:
print(s.imag)

[ 0.         0.         1.5259462 -1.5259462  0.         0.       ]


In [18]:
F = sym.simplify(sym.Matrix(A - B@K))
x0 = np.array([[0],[0],[0],[0],[0],[0]])
t = sym.Symbol('t')
sym.exp(-F*t)

NotImplementedError: Exponentiation is implemented only for matrices for which the Jordan normal form can be computed